# Optimalizace vs benchmarking
Na začátek je dobré projít čeho vlastně chceme dosáhnout.

Jednoduše: rychlosti

Optimalizace pomocí profilování nám umožní rozebrat současný kód, zjistit kde jsou slabiny (místa která zabírají nejvíce času) a tyto části optimalizovat=zrychlit.

Jedním trendem, na který při implementaci algoritmů narazíte poměrně často je, že většinou existuje více různých postupů jak něco naimplementovat. A často jsou všechny užitečné, neboť pro různá data/velikosti úloh se hodí různé přístupy. Toto pro nás rozhodne benchmarkování.

## Obecně k Profilování

Pro python existuje obrovské množství profilovacích nástrojů. A mnoho z nich je stále také v aktivním vývoji. To proto, že je mnoho funkcionalit, které je třeba profilovat:
- rychlost (časová náročnost)
- množství použité paměti
- paměťová propustnost = množství kopírovaných dat
- GPU profilování (celá další sada parametrů, zde se tímto nebudeme zabývat)


Profilery navíc přinášejí zpomalení samotného běhu kódu a tím i automatické zkreslení výsledků. Toto zpomalení může být od 1.x krát přes 2x - 5x až po 100x v některých případech.

## Základní profilování: Cprofile
jedná se o základní vestavěný profilovací nástroj. Počítá jednotlvá **volání** funkcí a jejich trvání.

In [ ]:
def heavy_calc(X):
    Y = X.copy()
    for i in range(10):
        Y = Y**i
    return Y

def heavy_loop(inputs):
    res = []
    for X in inputs:
        res.append(heavy_calc(X))
    return res

def code_setup():
    from numpy.random import rand
    N = 20
    M = 1000
    print("Will generate {} random arrays".format(N))
    inputs = [rand(M, M) for n in range(N)]
    print("Will calculate now")
    result = heavy_loop(inputs)
    print("Finished calculation")

In [ ]:
import cProfile
cProfile.run('code_setup()', 'pstats')

In [ ]:
from pstats import Stats
p = Stats('pstats')
p.print_stats()

No v tomhle se asi nevyznám.

Zkusme si to seřadit podle celkového času běhu `cumulative` odstraňme všechny cesty. Dále můžeme vypsat pouze prvních n (např. zde 10) nejvíce časově náročných funkcí.

In [ ]:
p.strip_dirs().sort_stats('cumulative').print_stats(10)

Všechno lze v jupyteru udělat pěkně pomocí magic příkazu:


In [ ]:
%prun -s cumulative -l 10 code_setup()

## Řádkové profilování: Line_profiler
V minulém příkladu jsme viděli, že základní profiler časuje pouze celé funkce. Co když chceme zjistit, které řádky kódu jsou nejvíce časově náročné?

Toto mnoho profilerů umožňuje, například `line_profiler`.

In [ ]:
# jedná se o knihovnu, je třeba ji nainstalovat
#!pip install line_profiler

Line profiler se dá spouštět z příkazového řádku pro celý file, ale to kdyžtak nechám zájemcům.

My si zde ukážeme jeho použití v Jupyteru pomocí magic příkazu.

In [ ]:
# je třeba jej naimportovat
%load_ext line_profiler

Spouští se příkazem `%lprun -f funkce_k_profilovani funkce_k_profilovani(parametry)`
- parametr `-f` určuje funkci, kterou chceme profilovat (může jich být více)
- parametr `-u` určuje jednotku času ve výstupu

In [ ]:
%lprun -u 1e-6 -f code_setup -f heavy_loop -f heavy_calc code_setup()

## Moderní, rychlý all-in-one profiler: Scalene
Scalene oproti předchozím umožňuje profilovat jak řádky tak funkce a navíc přidává i profilování paměti. (A GPU pokud používáme)

Vyžaduje instalaci pomocí `pip install scalene`

Spouští se příkazem `scalene script.py`

In [ ]:
%%writefile code_profile_test.py
def heavy_calc(X):
    Y = X.copy()
    for i in range(10):
        Y = Y**i
    return Y

def heavy_loop(inputs):
    res = []
    for X in inputs:
        res.append(heavy_calc(X))
    return res

def code_setup():
    from numpy.random import rand
    N = 20
    M = 1000
    print("Will generate {} random arrays".format(N))
    inputs = [rand(M, M) for n in range(N)]
    print("Will calculate now")
    result = heavy_loop(inputs)
    print("Finished calculation")

code_setup()

In [ ]:
!scalene code_profile_test.py